# Implementation verification: Toxicokinetics

In [22]:
using Revise
@time using DEBBase
using Plots, Plots.Measures, StatsPlots
using DataFrames
using BenchmarkTools

ERROR: LoadError: 

SystemError: opening file "c:\\Users\\simon\\Documents\\Julia\\DEBBase.jl\\src\\Macros.jl": No such file or directory
Stacktrace:
  [1] systemerror(p::String, errno::Int32; extrainfo::Nothing)
    @ Base .\error.jl:176
  [2] #systemerror#82
    @ .\error.jl:175 [inlined]
  [3] systemerror
    @ .\error.jl:175 [inlined]
  [4] open(fname::String; lock::Bool, read::Nothing, write::Nothing, create::Nothing, truncate::Nothing, append::Nothing)
    @ Base .\iostream.jl:293
  [5] open
    @ .\iostream.jl:275 [inlined]
  [6] open(f::Base.var"#418#419"{String}, args::String; kwargs::Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
    @ Base .\io.jl:393
  [7] open
    @ .\io.jl:392 [inlined]
  [8] read
    @ .\io.jl:473 [inlined]
  [9] _include(mapexpr::Function, mod::Module, _path::String)
    @ Base .\loading.jl:1959
 [10] include(mod::Module, _path::String)
    @ Base .\Base.jl:457
 [11] include(x::String)
    @ DEBBase c:\Users\simon\Documents\Julia\DEBBase.jl\src\DEBBase.jl:1

ErrorException: Failed to precompile DEBBase [0e91d03b-6562-4ef9-a38e-775ced00eaf8] to "C:\\Users\\simon\\.julia\\compiled\\v1.9\\DEBBase\\jl_B95B.tmp".

In [7]:
glb = GlobalBaseParams()

UndefVarError: UndefVarError: `GlobalBaseParams` not defined

In [15]:
u = [1., [2., 3.]]
statevars = [:X, :Y]
sttypes = [Float64, Vector{Float64}]

for (i, u_i) in enumerate(u)
    eval(:($(statevars[i]) = $(sttypes[i])($(u_i))))
end

In [16]:
Y

2-element Vector{Float64}:
 2.0
 3.0

## Simulating TK for a single stressor

## Varying the number of stressors

## Varying coupled TK parameter

## Uncoupling the TK parameter

## Modifying the model: Simplified TK

We start by having a look at the predefined TK function:

In [39]:
less(DEBBase.Ddot!)

function Ddot!(
    deb::AbstractParams,
    z::Int64; 
    C_W::Float64, 
    D::Float64, 
    S::Float64, 
    Sdot::Float64, 
    S_max::Float64
    )
    # surface area / volume ratio
    let AV_ratio = S^(2/3) / S, AV_max_ratio = S_max^(2/3) / S_max
        # TK with account for body size: smaller A/V ration = higher elimination rate, diluation by growth
        return deb.k_D[z] * (AV_max_ratio / AV_ratio) * (C_W[z] - D[z]) - D[z] * (Sdot/S)
    end
end

"""
Definition of reserveless DEB derivatives. 
$(TYPEDSIGNATURES)
"""
function DEB!(du, u, p, t)
    glb, deb = p
    # unpack scalar state variables
    X_p, X_emb, S, H, R = u

    C_W = u[glb.num_scalar_statevars+1:Int(glb.num_scalar_statevars+(length(u)-glb.num_scalar_statevars)/glb.num_vector_statevars)]
    D = u[Int(glb:num_scalar_statevars+1+length(C_W)):end]

    S = max(0, S) # control for negative values

    life_stage = determine_life_stage(deb; H = H, X_emb = X_emb)
    
    ddot 
    for (z,C_Wz) in enumerate(C_W)

In [30]:
function DEBBase.Ddot!()

end

In [45]:
less(DEBBaseParams())

MethodError: MethodError: no method matching DEBBaseParams()

In [47]:
[Float64, Int64]

2-element Vector{DataType}:
 Float64
 Int64

In [75]:
statevars = [:X_p, :X_emb, :S, :C_W]
sttypes = [Float64, Float64, Float64, Vector{Float64}]
u = [0.33, 2., 3., [4., 5., 314.]]

4-element Vector{Any}:
 0.33
 2.0
 3.0
  [4.0, 5.0, 314.0]

In [79]:
using DEBBase

In [81]:
DEBBase.@unpack u 

LoadError: LoadError: MethodError: no method matching var"@unpack"(::LineNumberNode, ::Module)

Closest candidates are:
  var"@unpack"(::LineNumberNode, ::Module, !Matched::Any)
   @ UnPack C:\Users\simon\.julia\packages\UnPack\EkESO\src\UnPack.jl:92

in expression starting at c:\Users\simon\Documents\Julia\DEBBase.jl\notebooks\ImplementationVerification\ImplemenationVerification_20231222_TK.ipynb:1

In [77]:
@unpack u

In [78]:
C_W

3-element Vector{Float64}:
   4.0
   5.0
 314.0

In [41]:
less(DEBBaseParams())

MethodError: MethodError: no method matching DEBBaseParams()